In [1]:
import pandas as pd
import os
import numpy as np

csv_path = "/Users/shantanughosh/Desktop/Shantanu_MS/Research/Mattia_Prosperi/Propensity_Dropout/Dataset/ihdp_sample.csv"

In [2]:
from dataloader import DataLoader
dL = DataLoader()
split_size = 0.8
np_covariates_X_train, np_covariates_X_test, np_covariates_Y_train, np_covariates_Y_test = dL.preprocess_data_from_csv(csv_path, split_size=0.8)
ps_train_set = dL.convert_to_tensor(np_covariates_X_train, np_covariates_Y_train)


.. Data Loading ..
ps_np_covariates_X: (747, 27)
ps_np_treatment_Y: (747, 1)
np_covariates_X_train: (597, 27)
np_covariates_Y_train: (597, 1)
------------------------------------------------------------
np_covariates_X_test: (150, 27)
np_covariates_Y_test: (150, 1)


In [5]:
from Propensity_socre_network import Propensity_socre_network
from Utils import Utils

device = Utils.get_device()
ps_net = Propensity_socre_network()
eval_parameters = {
        "eval_set": ps_train_set,
        "model_path": "./Propensity_Model/PS_model_epoch_250_lr_0.001.pth"
}
ps_score_list = ps_net.eval(eval_parameters, device, phase="eval")

# load data for ITE network
df_X = dL.prepare_tensor_for_ITE(np_covariates_X_train, np_covariates_Y_train,
                              ps_score_list)

.. Evaluation started ..
ps_np_covariates_X: (597, 27)
ps_np_treatment_Y: (597, 1)
Big X: (597, 29)


In [6]:
df_X

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-1.218158,-1.002474,1.128554,1.203011,0.058500,-0.526556,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.099737,5.703020,0.0,2.925839e-06
1,1.073153,0.196818,-0.733261,-0.879606,-0.129052,-1.023402,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2.730491,6.811552,0.0,3.892816e-07
2,-0.850683,-0.202946,-0.360898,-0.879606,2.059051,-1.023402,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.170285,4.361826,0.0,5.807775e-04
3,0.359820,0.596582,-1.105624,-0.879606,1.121292,0.963985,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,6.461459,3.122048,1.0,9.731616e-01
4,-0.980380,-0.202946,0.756191,-0.879606,0.808706,0.632754,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.734473,6.967970,0.0,5.243747e-03
5,-1.066845,-1.802002,1.128554,1.203011,0.683672,1.295216,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,5.430931,6.861205,0.0,8.557142e-05
6,0.251740,1.396110,-0.360898,0.161703,0.183534,1.295216,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.276305,4.293836,0.0,2.673906e-03
7,1.222304,2.195638,-1.477987,-0.879606,-0.441638,1.129600,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.543326,7.450607,0.0,2.263627e-12
8,0.208507,0.196818,-1.105624,2.244320,0.246052,0.798369,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,4.708913,5.498696,0.0,3.593406e-11
9,-0.656138,-1.002474,-0.360898,0.161703,0.246052,-0.857787,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.623305,5.705248,0.0,2.766633e-16


In [10]:
df_control = df_X[df_X.iloc[:, -2] == 0]

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-1.218158,-1.002474,1.128554,1.203011,0.058500,-0.526556,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.099737,5.703020,0.0,2.925839e-06
1,1.073153,0.196818,-0.733261,-0.879606,-0.129052,-1.023402,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2.730491,6.811552,0.0,3.892816e-07
2,-0.850683,-0.202946,-0.360898,-0.879606,2.059051,-1.023402,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.170285,4.361826,0.0,5.807775e-04
4,-0.980380,-0.202946,0.756191,-0.879606,0.808706,0.632754,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.734473,6.967970,0.0,5.243747e-03
5,-1.066845,-1.802002,1.128554,1.203011,0.683672,1.295216,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,5.430931,6.861205,0.0,8.557142e-05
6,0.251740,1.396110,-0.360898,0.161703,0.183534,1.295216,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.276305,4.293836,0.0,2.673906e-03
7,1.222304,2.195638,-1.477987,-0.879606,-0.441638,1.129600,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.543326,7.450607,0.0,2.263627e-12
8,0.208507,0.196818,-1.105624,2.244320,0.246052,0.798369,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,4.708913,5.498696,0.0,3.593406e-11
9,-0.656138,-1.002474,-0.360898,0.161703,0.246052,-0.857787,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.623305,5.705248,0.0,2.766633e-16
10,-0.007654,-0.202946,0.756191,0.161703,0.246052,-0.029709,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.649684,5.727380,0.0,5.919167e-03


In [17]:
df_treated = df_X[df_X.iloc[:, -2] == 1]
df_treated

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
3,0.359820,0.596582,-1.105624,-0.879606,1.121292,0.963985,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,6.461459,3.122048,1.0,0.973162
24,0.403053,-0.202946,0.011465,-0.879606,-0.066534,0.135907,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.817536,1.443363,1.0,0.999605
25,-1.931490,-1.802002,1.873280,-0.879606,-0.941775,-0.526556,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.255480,1.218525,1.0,0.962397
26,0.165275,0.596582,0.756191,-0.879606,0.808706,1.295216,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,7.247997,5.006365,1.0,0.994347
30,-1.239774,-1.402238,1.128554,-0.879606,0.933741,1.129600,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,6.473671,1.405829,1.0,0.905176
46,0.986688,0.996346,-1.105624,2.244320,1.121292,-0.029709,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,6.205641,3.145292,1.0,0.993616
48,-1.823409,-2.201766,1.873280,-0.879606,-0.191569,0.632754,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,6.947153,2.398184,1.0,0.990607
50,0.640830,0.596582,-0.360898,1.203011,-0.941775,-0.360940,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.857319,2.743203,1.0,0.999779
52,1.138001,0.996346,0.756191,0.161703,-0.629189,0.135907,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.855671,1.856800,1.0,0.999638
53,0.489517,0.596582,-0.733261,0.161703,1.058775,0.467138,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,4.974808,1.495454,1.0,0.970695


In [14]:
df_treated_X = df_treated.iloc[:, 0:25]
ps_score = df_treated.iloc[:, -1]
df_treated_Y_f = df_treated.iloc[:, -4]
df_treated_Y_cf = df_treated.iloc[:, -3]

In [19]:
df__Y_cf

3      3.122048
24     1.443363
25     1.218525
26     5.006365
30     1.405829
46     3.145292
48     2.398184
50     2.743203
52     1.856800
53     1.495454
68     0.953151
72     0.624132
74     3.295823
79     1.325607
81     0.812618
90     1.688818
91     4.451008
95     1.625704
98     2.253442
101    2.388555
112    2.198745
113    3.676856
114    0.604496
116    2.511192
118    1.486064
119    2.207291
122    3.928661
137    2.361597
143    0.632478
147    3.907360
         ...   
428    0.739044
432    2.536055
434    1.520922
437    2.598882
447    2.273142
450    2.482442
453    1.111920
462    3.452962
466    1.857130
473    4.481380
474    3.687381
492    5.316462
495    3.757283
514    2.446348
532    0.247619
538    3.066040
545    1.659699
546    2.951806
548    6.849375
556    2.821595
557   -0.626644
559    1.037790
561    6.194854
563    1.520137
572    4.082719
576    1.747822
580    2.577537
583    0.362493
587    9.538550
594    3.210349
Name: 26, Length: 112, d

In [21]:
import math
math.log(14)

2.6390573296152584

In [18]:
df_treated_Y_f

3      6.461459
24     7.817536
25     6.255480
26     7.247997
30     6.473671
46     6.205641
48     6.947153
50     6.857319
52     5.855671
53     4.974808
68     6.184087
72     5.977712
74     6.423668
79     6.228317
81     6.084255
90     7.690024
91     6.986570
95     6.468274
98     6.801466
101    6.956115
112    5.238206
113    7.713109
114    5.599927
116    6.178164
118    4.219257
119    6.507681
122    5.861061
137    5.042680
143    6.215572
147    9.314615
         ...   
428    5.539232
432    5.956705
434    5.894418
437    7.065181
447    6.120321
450    7.630186
453    7.513419
462    7.484104
466    7.636709
473    5.042879
474    7.485830
492    7.340269
495    7.351989
514    4.748230
532    6.296402
538    7.143979
545    7.860724
546    6.023766
548    6.270642
556    6.691096
557    5.327301
559    7.587765
561    7.517206
563    7.331502
572    5.418291
576    5.917606
580    6.057242
583    4.944220
587    8.204987
594    6.698660
Name: 25, Length: 112, d

In [16]:
ps_score

3      0.973162
24     0.999605
25     0.962397
26     0.994347
30     0.905176
46     0.993616
48     0.990607
50     0.999779
52     0.999638
53     0.970695
68     0.993375
72     0.987424
74     0.989740
79     0.942958
81     0.997544
90     0.994725
91     0.996620
95     0.969353
98     0.998976
101    0.997232
112    0.962657
113    0.999310
114    0.982033
116    0.993355
118    0.973984
119    0.977147
122    0.999939
137    1.000000
143    0.945951
147    0.986168
         ...   
428    0.992779
432    0.993605
434    0.972956
437    0.999998
447    0.997662
450    0.998962
453    0.945125
462    0.950801
466    0.993827
473    0.998850
474    0.995946
492    0.981486
495    0.925804
514    0.999701
532    0.993714
538    0.989962
545    0.997405
546    0.999861
548    0.994718
556    0.997597
557    0.981997
559    1.000000
561    0.999963
563    0.977351
572    0.984295
576    0.997489
580    0.999924
583    0.975076
587    0.988060
594    0.986214
Name: 28, Length: 112, d

In [26]:
import math
p = 0.005683999042958021
entropy = -(p*math.log(p)) - ((1-p)*math.log(1 - p))
entropy

0.035054659226622195

In [15]:
df_treated_X

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
3,0.359820,0.596582,-1.105624,-0.879606,1.121292,0.963985,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
24,0.403053,-0.202946,0.011465,-0.879606,-0.066534,0.135907,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,-1.931490,-1.802002,1.873280,-0.879606,-0.941775,-0.526556,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.165275,0.596582,0.756191,-0.879606,0.808706,1.295216,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
30,-1.239774,-1.402238,1.128554,-0.879606,0.933741,1.129600,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
46,0.986688,0.996346,-1.105624,2.244320,1.121292,-0.029709,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
48,-1.823409,-2.201766,1.873280,-0.879606,-0.191569,0.632754,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
50,0.640830,0.596582,-0.360898,1.203011,-0.941775,-0.360940,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
52,1.138001,0.996346,0.756191,0.161703,-0.629189,0.135907,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.489517,0.596582,-0.733261,0.161703,1.058775,0.467138,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
from torch.distributions import Bernoulli
import torch
input1 = torch.randn(5, 4, dtype=torch.double)
mask = Bernoulli(torch.full_like(input1, 0.6)).sample()/0.6
mask

tensor([[0.0000, 0.0000, 1.6667, 1.6667],
        [1.6667, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.6667, 1.6667, 1.6667],
        [1.6667, 1.6667, 0.0000, 1.6667],
        [1.6667, 1.6667, 0.0000, 0.0000]], dtype=torch.float64)

In [52]:
from DCN import DCN
network = DCN(training_flag=True).to(device)

In [58]:
network.shared1.weight

Parameter containing:
tensor([[ 0.1210, -0.1231, -0.0668,  ..., -0.1521, -0.0861, -0.0633],
        [ 0.1120,  0.1341,  0.0300,  ...,  0.1122,  0.0414, -0.0871],
        [ 0.0411, -0.1175,  0.0221,  ...,  0.1175, -0.1399,  0.1563],
        ...,
        [-0.1135, -0.0188, -0.0265,  ...,  0.0784,  0.1273,  0.1300],
        [ 0.0301, -0.0399, -0.1229,  ...,  0.0666, -0.0989, -0.1003],
        [ 0.0237, -0.1303, -0.0189,  ...,  0.0199, -0.0516,  0.0537]],
       requires_grad=True)

In [59]:
network.shared1.weight.requires_grad=False
network.shared1.weight

Parameter containing:
tensor([[ 0.1210, -0.1231, -0.0668,  ..., -0.1521, -0.0861, -0.0633],
        [ 0.1120,  0.1341,  0.0300,  ...,  0.1122,  0.0414, -0.0871],
        [ 0.0411, -0.1175,  0.0221,  ...,  0.1175, -0.1399,  0.1563],
        ...,
        [-0.1135, -0.0188, -0.0265,  ...,  0.0784,  0.1273,  0.1300],
        [ 0.0301, -0.0399, -0.1229,  ...,  0.0666, -0.0989, -0.1003],
        [ 0.0237, -0.1303, -0.0189,  ...,  0.0199, -0.0516,  0.0537]])

In [60]:
network.shared1.weight.requires_grad=True
network.shared1.weight

Parameter containing:
tensor([[ 0.1210, -0.1231, -0.0668,  ..., -0.1521, -0.0861, -0.0633],
        [ 0.1120,  0.1341,  0.0300,  ...,  0.1122,  0.0414, -0.0871],
        [ 0.0411, -0.1175,  0.0221,  ...,  0.1175, -0.1399,  0.1563],
        ...,
        [-0.1135, -0.0188, -0.0265,  ...,  0.0784,  0.1273,  0.1300],
        [ 0.0301, -0.0399, -0.1229,  ...,  0.0666, -0.0989, -0.1003],
        [ 0.0237, -0.1303, -0.0189,  ...,  0.0199, -0.0516,  0.0537]],
       requires_grad=True)

In [61]:
import torch.optim as optim
optimizer = optim.Adam(network.shared1.parameters(), lr=0.001)

In [68]:
next(network.shared1.parameters())

Parameter containing:
tensor([[ 0.1210, -0.1231, -0.0668,  ..., -0.1521, -0.0861, -0.0633],
        [ 0.1120,  0.1341,  0.0300,  ...,  0.1122,  0.0414, -0.0871],
        [ 0.0411, -0.1175,  0.0221,  ...,  0.1175, -0.1399,  0.1563],
        ...,
        [-0.1135, -0.0188, -0.0265,  ...,  0.0784,  0.1273,  0.1300],
        [ 0.0301, -0.0399, -0.1229,  ...,  0.0666, -0.0989, -0.1003],
        [ 0.0237, -0.1303, -0.0189,  ...,  0.0199, -0.0516,  0.0537]],
       requires_grad=True)

In [62]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [47]:
mask = Bernoulli(torch.full_like(input1, 0.4)).sample()/0.4
mask

tensor([[0.0000, 0.0000, 0.0000, 2.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 2.5000, 2.5000, 2.5000, 2.5000, 2.5000, 2.5000],
        [0.0000, 2.5000, 0.0000, 2.5000, 2.5000, 0.0000, 2.5000],
        [2.5000, 0.0000, 0.0000, 0.0000, 2.5000, 0.0000, 2.5000],
        [0.0000, 2.5000, 0.0000, 0.0000, 2.5000, 2.5000, 0.0000]],
       dtype=torch.float64)

In [48]:
from torch.autograd import Variable
import torch
input1 = torch.randn(5, 4, dtype=torch.double)
mask = Variable(torch.bernoulli(input1.data.new(input1.data.size()).fill_(0.5)))


In [49]:
mask = Variable(torch.bernoulli(input1.data.new(input1.data.size()).fill_(0.4)))/0.6
mask

tensor([[1.6667, 1.6667, 1.6667, 0.0000],
        [0.0000, 0.0000, 1.6667, 0.0000],
        [0.0000, 1.6667, 1.6667, 0.0000],
        [0.0000, 1.6667, 1.6667, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]], dtype=torch.float64)

In [43]:
mask = Variable(torch.bernoulli(input1.data.new(input1.data.size()).fill_(0.4)))/0.6
mask

tensor([[0.0000, 0.0000, 0.0000, 1.6667, 1.6667, 0.0000, 0.0000],
        [1.6667, 0.0000, 1.6667, 0.0000, 0.0000, 1.6667, 0.0000],
        [0.0000, 1.6667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.6667, 0.0000, 1.6667, 1.6667, 1.6667],
        [0.0000, 1.6667, 0.0000, 1.6667, 1.6667, 0.0000, 1.6667]],
       dtype=torch.float64)

In [77]:

import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

# 0) Prepare data
X_numpy, y_numpy = datasets.make_regression(n_samples=100, n_features=1, noise=20, random_state=4)

# cast to float Tensor
X = torch.from_numpy(X_numpy.astype(np.float32))
y = torch.from_numpy(y_numpy.astype(np.float32))
y = y.view(y.shape[0], 1)

n_samples, n_features = X.shape

# 1) Model
# Linear model f = wx + b
input_size = n_features
output_size = 1

class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, 5) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(5, output_size)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, output_size)
# model = nn.Linear(input_size, output_size)


# 2) Loss and optimizer
learning_rate = 0.01

criterion = nn.MSELoss()
print(model.l1.weight)
print(model.l1.bias)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# 3) Training loop
num_epochs = 2
for epoch in range(num_epochs):
    # Forward pass and loss
    if(epoch ==0):
        model.l1.weight.requires_grad=True
        model.l1.bias.requires_grad=False
    if(epoch ==1):
        model.l1.weight.requires_grad=True
        model.l1.bias.requires_grad=True
    
    y_predicted = model(X)
    loss = criterion(y_predicted, y)
    
    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
    print(model.l1.weight)
    print(model.l1.bias)

# Plot
predicted = model(X).detach().numpy()
# predicted


Parameter containing:
tensor([[-0.6345],
        [ 0.4527],
        [ 0.9107],
        [ 0.9035],
        [ 0.5843]], requires_grad=True)
Parameter containing:
tensor([-0.7151, -0.2662,  0.5141, -0.6998,  0.5972], requires_grad=True)
epoch: 1, loss = 5624.7729
Parameter containing:
tensor([[-0.6255],
        [ 0.6676],
        [ 0.7237],
        [ 1.0752],
        [ 0.5350]], requires_grad=True)
Parameter containing:
tensor([-0.7151, -0.2662,  0.5141, -0.6998,  0.5972])
epoch: 2, loss = 5460.4639
Parameter containing:
tensor([[-0.6586],
        [ 1.0212],
        [ 1.2973],
        [ 1.4532],
        [ 1.1962]], requires_grad=True)
Parameter containing:
tensor([-0.6952, -0.0082,  0.8533, -0.4477,  0.8044], requires_grad=True)
